# Лабораторная работа №2
*(Вариант 4)*

## Задание 1

По условию нам даны две выборки

$X_1\sim N(\mu_1, \sigma_1^2)$

$X_2 \sim N(\mu_2, \sigma_2^2)$

с известными параметрами $\mu_1 = 0, \mu_2 = 0, \sigma_1^2 = 2, \sigma_2^2 = 1$.

Необходимо оценить следующую функцию: $τ = \frac{\sigma_1^2}{\sigma_2^2}$.

Нормальное распределение имеет следующую связь с распределением Хи-Квадрат:

$U = \sum_{i=1}^{k}{(\frac{X_i - \mu_1}{\sigma_1})^2} \sim \chi^2(k) $
 при условии что $X_i$ - случайные величины, подчиняющиеся нормальному распределению $N(\mu_1, \sigma_1^2)$  

От распределения $ \sim \chi^2 $ мы можем перейти к распределению Фишера по следующему свойству:

Если $U_1 \sim \chi^2(k_1)$ и $U_2 \sim \chi^2(k_2)$, то случайная величина
$F = \frac{U_1/k_1}{U_2/k_2}$ имеет распределение Фишера со степенями свободы $(k_1, k_2)$

Воспользовавшись этими свойствами, получим следующее выражение:

$\frac{n_2 \cdot \sum_{i=1}^{k}{(X_1i - \mu_1)^2}}{n_1 \cdot \sum_{i=1}^{k}{(X_2i - \mu_2)^2}} ⋅ \frac{1}{τ} \sim F(n_1, n_2)$

Формула для поиска доверительного интервала:

$P(Z_{α/2} \le \frac{n_2 \cdot \sum_{i=1}^{k}{(X_1i - \mu_1)^2}}{n_1 \cdot \sum_{i=1}^{k}{(X_2i - \mu_2)^2}} ⋅ \frac{1}{τ} \sim F(n_1, n_2) \le Z_{1-α/2})  \ge 1 - α$

Выразим из этой формулы $τ$ :

$τ ∈ [ \frac{n_2 \cdot \sum_{i=1}^{k}{(X_1i - \mu_2)^2}}{Z_{1 - α/2} \cdot n_1 \cdot \sum_{i=1}^{k}{(X_2i - \mu_1)^2}}; \frac{n_2 \cdot \sum_{i=1}^{k}{(X_1i - \mu_2)^2}}{Z_{α/2} \cdot n_1 \cdot \sum_{i=1}^{k}{(X_2i - \mu_1)^2}}]$

### Программная часть

In [1]:
#Входные данные + import

import numpy as np
from scipy.stats import f

alpha = 0.05
N = 1000
mean1 = mean2 = 0
var1 = 2
var2 = 1
sd1 = np.sqrt(var1)
sd2 = np.sqrt(var2)
tau = var1 / var2

In [2]:
def count(size):
    n1 = n2 = size

    # Генерация двух выборок из генеральной совокупности, подчиненной нормальному распределению
    selection1 = np.random.normal(mean1, sd1, (N, n1))
    selection2 = np.random.normal(mean2, sd2, (N, n2))

    # Расчет доверительного интервала
    numerator = n2 * np.sum(((selection1 - mean1)**2), axis=1)
    denominator = n1 * np.sum((selection2 - mean2)**2, axis=1)
    fraction = numerator / denominator
    left = fraction / f.ppf(1 - alpha / 2, n1, n2) # f.ppf - функция для поиска квантиля
    right = fraction / f.ppf(alpha / 2, n1, n2)

    # Подсчет доверительных инетвалов, захвативших истинное значение
    counter = 0
    for x in (left <= tau) & (right >= tau):
        if x == True:
            counter += 1

    # Возвращаем вероятность, с которой интервал захватит истинное значение и среднюю длину интервала
    return counter / N, (right - left).mean()

In [3]:
# Проводим вычисления для выборок размером 25 и 10000 чисел

first_exp = count(25)
second_exp = count(10000)

print(f"--- Первый эксперимент(N = 25) ---\nВероятность покрытия - { first_exp[0] }\nСредняя длина доверительных интервалов - { first_exp[1] }\n")
print(f"--- Второй эксперимент(N = 10000) ---\nВероятность покрытия - { second_exp[0] }\nСредняя длина доверительных интервалов - { second_exp[1] }\n")

--- Первый эксперимент(N = 25) ---
Вероятность покрытия - 0.96
Средняя длина доверительных интервалов - 3.829598750808758

--- Второй эксперимент(N = 10000) ---
Вероятность покрытия - 0.947
Средняя длина доверительных интервалов - 0.15692460021652102



Анализируя результаты можно сказать, что количества элементов точность доверительных интевалов не зависит. Однако можно найти явные отличия в длине доверительных интервалов. Можно смело сказать что она пропорцианальна величине выборки, учавствующей в эксперименте, что на самом деле логично.

### Задание 2

Для построения асимптотически доверительного интервала уровня $1 - α$ для параметра $p$ геометрического распределения $Geom(p)$ при известном значении $p = 0.7$, можно воспользоваться ЦПТ и свойством ассимптотической нормальности геометрического распределения.

Формула доверительного интервала для нормального распределения имеет вид:

$\bar{x} - z_{\alpha/2} \frac{\sigma}{\sqrt{n}} \leq \mu \leq \bar{x} + z_{\alpha/2} \frac{\sigma}{\sqrt{n}}\$

Воспользуемся методом моментов. Для геометрического распределения $E[X] = \frac{1}{p}$. Тогда оценка параметра $p = \frac{1}{\bar{X}}$.

Применим ЦПТ, подставив параметры геометрического распределения, тогда интервал будет принимать вид:

$\bar{x} - z_{\alpha/2} \sqrt{\frac{s^2}{n}} \leq \frac{1}{p} \leq \bar{x} + z_{\alpha/2} \sqrt{\frac{s^2}{n}}$, где $s^2$ - выборочная дисперсия

Выразим параметр $p$:

$\frac{1}{\bar{x} + z_{\alpha/2} \sqrt{\frac{s^2}{n}}} \leq p \leq \frac{1}{\bar{x} - z_{\alpha/2} \sqrt{\frac{s^2}{n}}}$

Тогда

$p ∈ [\frac{1}{\bar{x} + z_{\alpha/2} \sqrt{\frac{s^2}{n}}}; \frac{1}{\bar{x} - z_{\alpha/2} \sqrt{\frac{s^2}{n}}}]$

In [4]:
#Входные данные + import

import numpy as np
from scipy.stats import norm

alpha = 0.05
p = 0.7
N = 1000

In [5]:
def count(size):

    # Генерация выборки из генеральной совокупности, подчиненной геометрическому распределению
    selection = np.random.geometric(p, (N, size))

    # Подсчет их средних выборочных
    sample_average = np.mean(selection, axis=1)

    # Подсчет стандартной ошибки
    se = np.std(selection, axis=1, ddof=1) / np.sqrt(size)

    # Поиск границ доверительного интервала
    left = 1 / (sample_average + norm.ppf(1 - alpha / 2, 0, 1) * se)
    right = 1 / (sample_average - norm.ppf(1 - alpha / 2, 0, 1) * se)

    # Подсчет доверительных инетвалов, захвативших истинное значение
    counter = 0
    for x in (left <= p) & (p <= right):
        if x == True:
            counter += 1

    # Возвращаем вероятность, с которой интервал захватит истинное значение и среднюю длину интервала
    return counter / N, np.mean(right - left)

In [5]:
# Проводим вычисления для выборок размером 25 и 10000 чисел

first_exp = count(25)
second_exp = count(10000)

print(f"--- Первый эксперимент(N = 25) ---\nВероятность покрытия - { first_exp[0] }\nСредняя длина доверительных интервалов - { first_exp[1] }\n")
print(f"--- Второй эксперимент(N = 10000) ---\nВероятность покрытия - { second_exp[0] }\nСредняя длина доверительных интервалов - { second_exp[1] }\n")

--- Первый эксперимент(N = 25) ---
Вероятность покрытия - 0.947
Средняя длина доверительных интервалов - 3.9652142392585894

--- Второй эксперимент(N = 10000) ---
Вероятность покрытия - 0.944
Средняя длина доверительных интервалов - 0.15690376339001494



По результатам эксперимента можно сказать, что ассимптотический доверительный интервал, увеличивает свю точность по мере увеличения выборки, что полностью соответствует его определению. Также зависимость длины интевала от объема выборки, описанная в первом задании, сохраняется.